In [26]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import pandas as pd
# from tensorflow import keras
# !pip install tensorflow-datasets
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# print(tf.__version__)

# !pip uninstall -y tensorflow keras tb-nightly keras-nightly tf-nightly
# !pip install tensorflow==2.15 keras==2.15

import tensorflow as tf

import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-08-17 21:42:15--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2025-08-17 21:42:16 (8.18 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-08-17 21:42:16--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [46]:
df_train = pd.read_csv(train_file_path,sep="\t",header=None, names = ['label', 'text'])
df_test = pd.read_csv(test_file_path,sep="\t",header=None, names = ['label', 'text'])

df_train.tail()

,label,text
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...
4178,ham,tell my bad character which u dnt lik in me. ...


In [47]:
# start preprocessing
import string

# 1. Convert labels to numeric
df_train['label'] = df_train['label'].apply(lambda x: 1 if x == 'spam' else 0)
df_test['label'] = df_test['label'].apply(lambda x: 1 if x == 'spam' else 0)

## 2. Define stopwords
stop_words = set(stopwords.words('english'))

# 3. Clean function
def clean_text(text):
    # remove punctuation with regex
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)
    # lowercase + split
    tokens = text.lower().split()
    # remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # join back into a string
    return " ".join(tokens)

# Apply to both train & test
df_train['clean_text'] = df_train['text'].apply(clean_text)
df_test['clean_text'] = df_test['text'].apply(clean_text)

In [52]:
df_train['label'].nunique()

2

In [38]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Convert text to padded sequences
# tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
# tokenizer.fit_on_texts(df_train['clean_text'])

# X_train = tokenizer.texts_to_sequences(df_train['clean_text'])
# X_test = tokenizer.texts_to_sequences(df_test['clean_text'])

# # Pad to equal length
# maxlen = 100
# X_train = pad_sequences(X_train, maxlen=maxlen, padding='post')
# X_test = pad_sequences(X_test, maxlen=maxlen, padding='post')

# y_train = df_train['label'].values
# y_test = df_test['label'].values


In [39]:
# model = Sequential()
# model.add(Embedding(input_dim=10000, output_dim=64, input_length=maxlen))
# model.add(LSTM(64, return_sequences=False))
# model.add(Dropout(0.3))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# model.summary()

# model.fit(X_train, y_train,
#           validation_data=(X_test, y_test),
#           epochs=5, batch_size=32)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 64)           640000    
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 675137 (2.58 MB)
Trainable params: 67513

In [48]:
# Feature extraction from text
vectorizer = TfidfVectorizer(max_features=5000)

X_train = vectorizer.fit_transform(df_train['clean_text'])
X_test = vectorizer.transform(df_test['clean_text'])

y_train = df_train['label']
y_test = df_test['label']


In [49]:
# build simple model architecture
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train.toarray(), y_train,
          validation_data=(X_test.toarray(), y_test),
          epochs=5, batch_size=32)


Epoch 1/5
131/131 [==============================] - 2s 9ms/step - loss: 0.3150 - accuracy: 0.8701 - val_loss: 0.1371 - val_accuracy: 0.9648
Epoch 2/5
131/131 [==============================] - 1s 8ms/step - loss: 0.0806 - accuracy: 0.9835 - val_loss: 0.0520 - val_accuracy: 0.9842
Epoch 3/5
131/131 [==============================] - 1s 11ms/step - loss: 0.0218 - accuracy: 0.9943 - val_loss: 0.0477 - val_accuracy: 0.9864
Epoch 4/5
131/131 [==============================] - 1s 7ms/step - loss: 0.0092 - accuracy: 0.9983 - val_loss: 0.0542 - val_accuracy: 0.9864
Epoch 5/5
131/131 [==============================] - 1s 7ms/step - loss: 0.0045 - accuracy: 0.9986 - val_loss: 0.0563 - val_accuracy: 0.9871


In [50]:
# # function to predict messages based on model
# # (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
# def predict_message(pred_text):
#     clean_msg = clean_text(pred_text)
#     seq = tokenizer.texts_to_sequences([clean_msg])
#     pad = pad_sequences(seq, maxlen=maxlen, padding='post')
#     prob = model.predict(pad)[0][0]
#     label = "spam" if prob >= 0.5 else "ham"
#     prediction = [float(prob), label]

#     return (prediction)

# pred_text = "how are you doing today?"

# prediction = predict_message(pred_text)
# print(prediction)


def predict_message(pred_text):
    clean_msg = clean_text(pred_text)
    vec = vectorizer.transform([clean_msg])
    prob = model.predict(vec.toarray())[0][0]
    label = "spam" if prob >= 0.5 else "ham"
    prediction = [float(prob), label]
    return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 44ms/step
[0.0009322069236077368, 'ham']


In [53]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 14ms/step
You passed the challenge. Great job!
